In [75]:
import pandas as pd
import igraph
import numpy as np
import pdb
from ndex2.nice_cx_network import NiceCXNetwork
import ndex2.client as nc
import ndex2
import networkx as nx
from fa2 import ForceAtlas2

#from https://github.com/idekerlab/cy-rest-R/blob/develop/workflow1_structure_based_visualization.R

# Step 1: Network Data Preparation

Load network from NDEx

In [76]:
# nice_cx_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='c060aff9-86d4-11e7-a10d-0ac135e8bacf')
nice_cx_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303')
# nice_cx_network =  ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='ff8e7def-6e09-11e9-848d-0ac135e8bacf')
nice_cx_network =  ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='38a9cebf-23dc-11e8-b939-0ac135e8bacf')

In [77]:
nice_cx_network.print_summary()

Name: BioGRID: Protein-Protein Interactions (HIV-1)
Nodes: 1127
Edges: 1771
Node Attributes: 3297
Edge Attributes: 12397



In [78]:
nice_cx_from_server_df=nice_cx_network.to_pandas_dataframe()

In [79]:
nice_cx_from_server_df.head()

,source,interaction,target
0,vpr,interacts-with,SF3B2
1,vpr,interacts-with,SF3B2
2,vpr,interacts-with,SF3B2
3,SF3B2,interacts-with,vpr
4,tat,interacts-with,CCNT1


Convert it to simple edge list

In [80]:
edgelist=nice_cx_from_server_df.iloc[:,[0,2]]

Convert DF to undirected igraph object
This is a PPI network, so import as undirected.

In [81]:
tuples=[tuple(x) for x in edgelist.values]
g_original = igraph.Graph.TupleList(tuples,directed=False)

Extract componentes (individual connected subgraphs)

In [82]:
subgraphs=g_original.decompose()

Pick largest subgraph

In [83]:
tmp=[i.vcount() for i in subgraphs]
largeset_subgraph = subgraphs[tmp.index(max(tmp))]

Remove duplicate edges

In [84]:
g = largeset_subgraph.simplify(multiple=True, loops=True)
g.name = "Yeast network"

# Step 2: Basic statistical analysis

Global Network Statics

In [85]:
print("Density:",g.density())
print("Transitivity:",g.transitivity_undirected())

Density: 0.0020614624720843625
Transitivity: 0.0010073794542561781


Node statistics

In [86]:
g_closeness = g.vs.closeness() # Closeness Centrarity
g_degree = g.vs.degree() # Degree
g_pagerank = g.vs.pagerank(directed = False) # PageRank
g_vs_betweenness = g.vs.betweenness() # Betweenness Centrarity

Edge statics

In [87]:
g_es_betweenness = g.es.edge_betweenness() # Edge Betweenness

# Step 3: Community Detection: Try multiple algorithms

Clustering

In [88]:
communities_greedy = g.community_fastgreedy().as_clustering()
communities_leading = g.community_leading_eigenvector()
communities_label_propagation = g.community_label_propagation()

In [89]:
v_community_greedy = communities_greedy.membership
v_community_leading = communities_leading.membership
v_community_label_propagation = communities_label_propagation.membership

In [90]:
def getCommunityEdge(g, community):
    num_edges = g.ecount()
    edge_community = [-1]*num_edges
    comms = community
    sources=[i for i, _ in g.get_edgelist()]
    targets=[j for _, j in g.get_edgelist()]
    for i in range(num_edges):
        sidx = sources[i]
        tidx = targets[i]
        source = comms[sidx]
        target = comms[tidx]
        
        if source == target:
            edge_community[i] = source
    return edge_community
    

In [91]:
e_commnity_greedy = getCommunityEdge(g, v_community_greedy)
e_commnity_leading = getCommunityEdge(g, v_community_leading)
e_community_label_propagation = getCommunityEdge(g, v_community_label_propagation)

## Community to Colors

In [92]:
import seaborn as sns

In [93]:
def communityToColors(members):
    basecolor='#AAAAAA'
    num_members=len(members)
    num_communities=max(members)+1
    colors = [basecolor]*num_members
    
    colorp = sns.color_palette("hls", num_communities)
    
#     sns.palplot(colorp)
    
    #print(num_communities,num_members)
    colorpalette=[rgb2hex(int(a[0]*255),int(a[1]*255),int(a[2]*255)) for a in colorp]
    
    for i in range(num_members):
        newcolor = colorpalette[members[i]]
        if( members[i] == -1 ):
            newcolor = basecolor
        colors[i] = newcolor
    return colors

def rgb2hex(r,g,b): #RGB配列からHTMLなどに使われる16進数表現へ
    # r , g , b = 0 〜 255 # int型
    color = (r, g , b)
    html_color = '#%02X%02X%02X' % (color[0],color[1],color[2])
    return html_color


In [94]:
g.vs['color'] = communityToColors(communities_greedy.membership)
g.es['color'] = communityToColors(e_commnity_greedy)

## Convert igraph to networkx

In [95]:
def search_interaction(df,src,tgt):
    if len(df[(df['source']==src) & (df['target']==tgt)]):
        return df[(df['source']==src) & (df['target']==tgt)]['interaction'].values[0]
    else:
        return df[(df['source']==tgt) & (df['target']==src)]['interaction'].values[0]

In [96]:
G_nx=nx.Graph()
for i in range(g.vcount()):
    G_nx.add_node(g.vs['name'][i])
for i,j in g.get_edgelist():
    G_nx.add_edge(g.vs['name'][i],g.vs['name'][j])

# py2cytoscape basic setup
## Setting up py2cytoscape client

Many of the Legacy APIs are deprecated.  Check this document as the reference:

https://py2cytoscape.readthedocs.io/en/latest/

### For native version users (running from host)
You don't have to specify IP since you are running both Jupyter and Cytoscape form the same host

### For Docker users
You shoulod check the IP address of your host system (i.e., workstation's IP) and pass it to the client.

In [97]:
from py2cytoscape import cyrest

# This should be changed to your host IP
host_ip = '192.168.1.47'
cytoscape=cyrest.cyclient(host=host_ip)
cytoscape.version()
cytoscape.status()

apiVersion v1
cytoscapeVersion 3.7.1
CyREST online!


In [98]:
# Delete current session and start a new one
cytoscape.session.new()

## Layout using by ForceAtlas2

In [99]:
import networkx as nx
from fa2 import ForceAtlas2
import matplotlib.pyplot as plt

# G = nx.random_geometric_graph(400, 0.2)

forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=True,  # Dissuade hubs
                        linLogMode=False,  # NOT IMPLEMENTED
                        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                        edgeWeightInfluence=1.0,

                        # Performance
                        jitterTolerance=1.0,  # Tolerance
                        barnesHutOptimize=True,
                        barnesHutTheta=1.2,
                        multiThreaded=False,  # NOT IMPLEMENTED

                        # Tuning
                        scalingRatio=20,
                        strongGravityMode=False,
                        gravity=10,

                        # Log
                        verbose=True)


g.es['weights']=[ 0 if i==-1 else 10 for i in e_commnity_greedy]
positions = forceatlas2.forceatlas2_igraph_layout(g, pos=None, iterations=2000,weight_attr='weights')
#positions_x = forceatlas2.forceatlas2_networkx_layout(G_nx, pos=None, iterations=2000)

#nx.draw_networkx_nodes(G_nx, positions_x, node_size=20, with_labels=False, node_color="blue", alpha=0.4)
#nx.draw_networkx_edges(G_nx, positions_x, edge_color="green", alpha=0.05)
#plt.axis('off')
#plt.show()


100%|██████████| 2000/2000 [00:26<00:00, 75.41it/s]

BarnesHut Approximation  took  10.42  seconds
Repulsion forces  took  12.87  seconds
Gravitational forces  took  0.26  seconds
Attraction forces  took  0.07  seconds
AdjustSpeedAndApplyForces step  took  1.44  seconds


### Convert "positions" to "certesian"

In [100]:


# positions = list(positions)

# certesian = [{'node':i,'x': float(positions[n][0]),'y': float(positions[n][1])}for i, n in enumerate(positions)]
certesian = [{'node':i,'x': list(positions)[i][0],'y': list(positions)[i][1]}for i in range(len(positions))]

In [101]:
ncx_from_x=ndex2.create_nice_cx_from_networkx(G_nx)

for i in range(g.vcount()):
    ncx_from_x.set_node_attribute(i, "closeness", g_closeness[i])
    ncx_from_x.set_node_attribute(i, "degree", g_degree[i])
    ncx_from_x.set_node_attribute(i, "pagerank", g_pagerank[i])
    ncx_from_x.set_node_attribute(i, "betweenness", g_vs_betweenness[i])
    ncx_from_x.set_node_attribute(i, "community.greedy", v_community_greedy[i])
    ncx_from_x.set_node_attribute(i, "community.leading", v_community_leading[i])
    ncx_from_x.set_node_attribute(i, "community.label.propagation", v_community_label_propagation[i])
    ncx_from_x.set_node_attribute(i, "colors.community.greedy", communityToColors(v_community_greedy)[i])
    ncx_from_x.set_node_attribute(i, "colors.community.leading", communityToColors(v_community_leading)[i])
    ncx_from_x.set_node_attribute(i, "colors.community.label.propagation", communityToColors(v_community_label_propagation)[i])
 


for i in range(g.ecount()):
    ncx_from_x.set_edge_attribute(i, "betweenness.edge", g_es_betweenness[i])
    ncx_from_x.set_edge_attribute(i, "community.greedy", e_commnity_greedy[i])
    ncx_from_x.set_edge_attribute(i, "community.leading", e_commnity_leading[i])
    ncx_from_x.set_edge_attribute(i, "community.label.propagation", e_community_label_propagation[i])
    ncx_from_x.set_edge_attribute(i, "colors.community.greedy", communityToColors(e_commnity_greedy)[i])
    ncx_from_x.set_edge_attribute(i, "colors.community.leading", communityToColors(e_commnity_leading)[i])
    ncx_from_x.set_edge_attribute(i, "colors.community.label.propagation", communityToColors(e_community_label_propagation)[i])
    


In [102]:
import json

with open('tmp.json') as f:
    tmp = json.load(f)




### Apply "certesian" to NiceCX

In [103]:
print(type(ncx_from_x.VISUAL_PROPERTIES))
# ncx_from_x.set_opaque_aspect("VISUAL_PROPERTIES",colors)

# ncx_from_x.apply_template('public.ndexbio.org','92084251-c859-11e9-aa81-0ac135e8bacf')

ncx_from_x.set_opaque_aspect("cartesianLayout", certesian)
ncx_from_x.set_opaque_aspect("cyVisualProperties",tmp['cyVisualProperties'])

<class 'str'>


## Send to Cytoscape

In [104]:
import requests

cxobj = ncx_from_x.to_cx()
cyrest_url = 'http://' + host_ip + ':1234/cyndex2/v1/networks/cx'
print(cyrest_url)
response = requests.post(cyrest_url, data=json.dumps(cxobj))
print(response.json())

Generating CX
http://192.168.1.47:1234/cyndex2/v1/networks/cx
{'data': {'suid': 21152, 'uuid': ''}, 'errors': []}


##  upload the network

In [105]:
my_account="am"
my_password="ndexpassword"
my_server="http://public.ndexbio.org"

if my_account == 'enter your username here':
    print('*******WARNING!!!!*******')
    print('Please change the username and password before proceeding')
else:
    try:
        my_ndex=nc.Ndex2(my_server, my_account, my_password)
        my_ndex.update_status()
        print("Success.  Please continue.")
    except Exception as inst:
        print("Could not access account %s with password %s" % (my_account, my_password))
        print(inst.args)

Success.  Please continue.


In [106]:
upload_message=ncx_from_x.upload_to(my_server, my_account, my_password)
print(upload_message)

Generating CX
http://public.ndexbio.org/v2/network/cb6d5b85-c957-11e9-aa81-0ac135e8bacf
